In [159]:
from sets import Set
import string

In [414]:
alphabet=dict(zip(string.letters,[ord(c)%32 for c in string.letters]))
alphabet_reverse = {}
for x in alphabet:
    alphabet_reverse[alphabet[x]] = x.upper()
    
# alphabet -> num
def letter_to_idx(l):
    return alphabet[l]

def idx_to_letter(idx):
    return alphabet_reverse[idx]

# build graph
def build_graph(input):
    info = []
    max_idx = 0
    for x in input.replace(' ', '').split(','):
        n_from = letter_to_idx(x[0]) 
        n_to = letter_to_idx(x[1])
        max_idx = max(n_from, n_to, max_idx)
        distance = int(x[2])
        info.append((n_from, n_to, distance))

    graph = [[0 for x in range(max_idx)] for r in range(max_idx)]
    for n_from, n_to, distance in info:
        graph[n_from - 1][n_to - 1] = distance

    return graph

def print_graph(graph):
    header = " ".join([' '] + [idx_to_letter(x + 1) for x in range(len(graph))])
    print header
    for ri in range(len(graph)):
        node_name = idx_to_letter(ri + 1)
        row = graph[ri]
        row_str = " ".join([node_name] + [str(x) if x != 0 else "0" for x in row])
        print row_str

# node_idx starts from 1
# returns [(node_idx, distance)]
def get_related_nodes(node_idx, graph):
    related_nodes = graph[node_idx - 1]
    return [(i + 1, related_nodes[i]) for i in range(len(related_nodes)) if related_nodes[i] != 0]

# find shortest path using dijkstra algorithm
def dijkstra(start, end, graph):
    
    # letter -> idx
    start = letter_to_idx(start)
    end = letter_to_idx(end)
    
    # {node_id: tent_distance}
    node_tentative_distances = {}
    # {node_id: prev_node_id}
    node_prev = {}
    visited_nodes = Set()

    # compute tentative distance for nodes
    def forward(current_node, current_node_tent_distance):
        # get related nodes
        related_nodes = get_related_nodes(current_node, graph=graph)
        
        # update node tentative distance
        for related_node_idx, distance_to_related_node in related_nodes:
            if related_node_idx not in visited_nodes:
                related_node_tentative_distance = node_tentative_distances.get(related_node_idx)
                if related_node_tentative_distance is None:
                    node_tentative_distances[related_node_idx] = current_node_tent_distance + distance_to_related_node
                    node_prev[related_node_idx] = current_node
                elif related_node_tentative_distance > current_node_tent_distance + distance_to_related_node:
                    # update tentative distance
                    node_tentative_distances[related_node_idx] = current_node_tent_distance + distance_to_related_node
                    node_prev[related_node_idx] = current_node

        # mark current node as visited
        if current_node_tent_distance != 0:
            # do not add start node
            visited_nodes.add(current_node)
        
        # recursion
        for related_node_idx, distance_to_related_node in related_nodes:
            if related_node_idx not in visited_nodes:
                related_node_tentative_distance = node_tentative_distances[related_node_idx]
                forward(related_node_idx, related_node_tentative_distance)
                
    # restore shortest path
    def reverse(node_idx, depth = 0):
        if node_idx == start:
            if start == end and depth == 0:
                # prevent early exit when start node eq end node
                return reverse(node_prev[node_idx], depth + 1) + [node_idx]
            else:
                return [node_idx]
        else:
            return reverse(node_prev[node_idx], depth + 1) + [node_idx]

        
    # run
    forward(start, 0)
    shortest_path = reverse(end)    
    shortest_distance = node_tentative_distances.get(end)
    return shortest_path, shortest_distance

def print_path(path, distance):
    path_str = "->".join(idx_to_letter(x) for x in path)
    return '[{}] {}'.format(distance, path_str)

# def find_shortest_route_distance(route_str, print_info = False):
#     route = route_str.split('-')
#     total_distance = 0
#     for i in range(len(route) - 1):
#         start, end = route[i], route[i+1]
#         shortest_path, distance = dijkstra(start=start, end=end, graph=graph)
#         total_distance += distance
#         if print_info:
#             print '{} -> {} {}'.format(start, end, distance)
#     return total_distance

# return None if not exist
def get_route_distance(route_str, graph):
    route = route_str.split('-')
    total_distance = 0
    for i in range(len(route) - 1):
        start_idx, end_idx = letter_to_idx(route[i]), letter_to_idx(route[i+1])
        distance = graph[start_idx - 1][end_idx - 1]
        if distance > 0:
            total_distance += distance
        else:
            return None
    return total_distance

In [415]:
input = "AB5, BC4, CD8, DC8, DE6, AD5, CE2, EB3, AE7"
graph = build_graph(input)
print_graph(graph)

  A B C D E
A 0 5 0 5 7
B 0 0 4 0 0
C 0 0 0 8 2
D 0 0 8 0 6
E 0 3 0 0 0


In [421]:
# find shortest path
shortest_path, distance = dijkstra(start='B', end='E', graph=graph)
print_path(shortest_path, distance)

'[6] B->C->E'

In [422]:
# find distance
distance = get_route_distance('A-E-B-C-D', graph)
if distance:
    print 'distance {}'.format(distance)
else:
    print 'NO SUCH ROUTE'


distance 22
